# Colab的默认环境跟这里的默认环境有差异，需要安装额外的依赖

In [1]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 118 not upgraded.
Need to get 3,316 kB of archives.
After this operation, 11.1 MB of additional disk space will be used.
Get:1 https://repo.huaweicloud.com/ubuntu focal/universe amd64 git-lfs amd64 2.9.2-1 [3,316 kB]
Fetched 3,316 kB in 0s (7,008 kB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package git-lfs.
(Reading database ... 45902 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.9.2-1_amd64.deb ...
Unpacking git-lfs (2.9.2-1) ...
Setting up git-lfs (2.9.2-1) ...


In [2]:
!pip install wget
!pip install transformers
!pip install bitsandbytes
!pip install jax
!apt-get update
!apt-get install -y p7zip-full
!apt-get install -y iputils-ping


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=83683a7625e4b4a3831b07b50755ec7546e7ca04c46026b1831392671748afe0
  Stored in directory: /root/.cache/pip/wheels/df/d8/7c/a62984d928c4d9d2d2abe86ae3bd091679cfc87bf6709256df
Successfully built wget
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 10.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 7.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.2/701.2 kB 6.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 kB 4.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.3/772.3 kB 5.7 MB/s eta 0:00:00a 0:00:01
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62

/root/autodl-tmp/OneClick-stable-diffusion


# 创建Temp文件夹

In [3]:
import wget
import urllib.request
import os,sys
sys.path.append('../') # 因为func与ipynb位于同一个目录下，所以要往上一层路径索引
from func.env import setProxyCLI
cb=setProxyCLI()
proxy=cb['proxy']
region=cb['region']
proxyURL=cb['proxyURL']


# 在当前运行的ipynb文件所在的目录下创建临时文件夹temp
def create_temp_folder(temp_folder):
  if not os.path.exists(temp_folder):
    os.makedirs(temp_folder)

# 获取当前运行的 ipynb 文件所在的目录
cwd = os.getcwd()

# 在该目录下创建临时文件夹 temp
temp_folder = os.path.join(cwd, 'temp')

# 如果temp文件夹不存在，则创建文件夹
create_temp_folder(temp_folder)

# 覆盖下载 fast-DreamBooth.ipynb 文件到 temp 文件夹中
file_name='fast-DreamBooth.ipynb'
fastDreamBoothPath = os.path.join(temp_folder, file_name)
if os.path.exists(fastDreamBoothPath):
    os.remove(fastDreamBoothPath)

proxy_handler = urllib.request.ProxyHandler({'http': proxyURL})
opener = urllib.request.build_opener(proxy_handler)
urllib.request.install_opener(opener)

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/fast-DreamBooth.ipynb",
    os.path.join(temp_folder, file_name)
)

print(fastDreamBoothPath)

Ping程序运行耗时: 6毫秒


ping: alchemist-experience: Name or service not known


/root/autodl-tmp/OneClick-stable-diffusion/dreambooth/temp/fast-DreamBooth.ipynb


# 清理Notebook中的输出，对Colab版中的路径进行替换，以适应openbayes的路径

In [65]:
import json
import os

def modify_file_name(file_path):
  # 获取文件名和文件扩展名
  file_name, file_ext = os.path.splitext(file_path)
  # 在文件名后加上 -modified
  modified_file_path = file_name + "-modified" + file_ext
  return modified_file_path

def replace_root_path(notebook: str, old_root: str, new_root: str) -> str:
    # 加载notebook
    with open(notebook, "r") as f:
        nb = json.load(f)

    # 隐藏代码
    for cell in nb["cells"]:
        if cell["cell_type"] == "code":
            cell["metadata"] = {"collapsed": True}

    # 清空notebook中所有输出
    for cell in nb['cells']:
        if cell['cell_type'] == 'code':
            # 清除代码单元格的执行次数
            cell['execution_count'] = None
            # 清除代码单元格的输出
            cell['outputs'] = []

    # 替换旧的根路径为新的根路径
    for cell in nb["cells"]:
        if cell["cell_type"] == "code":
            for i, line in enumerate(cell["source"]):
                # 添加一个条件判断，只有当该行不是网址路径时才进行替换
                if not line.startswith(("http", "https")):
                    cell["source"][i] = line.replace(old_root, new_root)
                # 把引入google.colab的库的代码行标注为注释
                if "google.colab" in line:
                    cell["source"][i] = "# " + line
                # 把使用google drive盘挂载的代码行标注为注释
                if "drive.mount" in line:
                    cell["source"][i] = "# " + line
                
                # # 原始字符串
                # original = "!cp -r /openbayes/home/content/usr/local/lib/python3.8/dist-packages /usr/local/lib/python3.8/"
                # if original in line:
                #     # 替换python路径
                #     cell["source"][i] = re.sub(r'/usr/local/lib/python3.8/', '/usr/local/lib/python/', original)
                    
    # 替换@param{type: 'xxx'}中的单引号为双引号
    for cell in nb["cells"]:
        if cell["cell_type"] == "code":
            for i, line in enumerate(cell["source"]):
                if "@param" in line:
                    cell["source"][i] = line.replace("'", '"')
                    
    # 保存修改后的notebook
    newNotebook=modify_file_name(notebook)
    with open(newNotebook, "w") as f:
        json.dump(nb, f)
        
    return newNotebook

def check_environment(output):
    # 检查输出是否包含字符串"autodl"
    # print(output)
    content_path=''
    for i in output:
        if "autodl" in i:
            print("当前运行环境为autodl")
            content_path = "/root/content"
            break
        # 检查输出是否包含字符串"openbayes"
        elif "openbayes" in i:
            print("当前运行环境为openbayes")
            content_path = "/openbayes/home/content"
            break
    return content_path

# 尝试执行第一个命令并存储输出
try:
    output1 = !cd /openbayes/home && /etc/welcome
    content_path = check_environment(output1)
except Exception as e:
    print("无法执行命令：", e)
    
# 尝试执行第二个命令并存储输出
try:
    output2 = !source /etc/autodl-motd
    content_path = check_environment(output2)
except Exception as e:
    print("无法执行命令：", e)
    
print("内容路径：", content_path)

# 在根目录下创建Content文件夹
content_folder = content_path
# 在当前运行的ipynb文件所在的目录下创建临时文件夹temp
if not os.path.exists(content_folder):
    os.makedirs(content_folder)

newNotebookPath=replace_root_path(
    notebook=fastDreamBoothPath,
    old_root="/content",
    new_root=content_folder
)
print(newNotebookPath)

当前运行环境为autodl
内容路径： /root/content
/root/autodl-tmp/OneClick-stable-diffusion/dreambooth/temp/fast-DreamBooth-modified.ipynb


# 对指定的变量名进行字符串和参数的替换

In [66]:
import json

def modify_file_name(file_path):
  # 获取文件名和文件扩展名
  file_name, file_ext = os.path.splitext(file_path)
  # 在文件名后加上 -modified
  modified_file_path = file_name + "-customData" + file_ext
  return modified_file_path

# 读取修改后的notebook
def update_ipynb_vars(ipynb_file, var_map):
  with open(ipynb_file, 'r') as f:
    notebook = json.load(f)
  
  # 用更新后的值替换变量赋值
  for cell in notebook['cells']:
    if cell['cell_type'] == 'code':
      for i in range(len(cell['source'])):
        for var, val in var_map.items():
          if isinstance(val, str):
            val = f'"{val}"'
          # 有 #@ 说明这一行是提供给用户自行填充的参数，所以才进行参数替换
          identifier="#@"
          thisLine=cell['source'][i]
          if identifier in thisLine and (thisLine.startswith(f'{var} =') or thisLine.startswith(f'{var}=')):
            annotation=thisLine.split(identifier)[1]
            cell['source'][i] = f'{var}={val} {identifier+annotation}' 

  # 保存修改后的notebook
  updatedDataNotebookPath=modify_file_name(ipynb_file)
  with open(updatedDataNotebookPath, 'w') as f:
    json.dump(notebook, f)

  return updatedDataNotebookPath

# 更新变量
replacements = {
  'Model_Version': 'V2.1-768px',
  'Session_Name': 'FuXingHao768',
  'Crop_size': 768,
  'Remove_existing_instance_images': False,
  'IMAGES_FOLDER_OPTIONAL':"/openbayes/home/content/gdrive/MyDrive/Fast-Dreambooth/Sessions/FuXingHao768/instance_images",
  'Crop_images':False,
  'Resolution': 768,
}

# 更新变量
updatedDataNotebookPath=update_ipynb_vars(newNotebookPath, replacements)

# 检查最终版本的ipynb里面的赋值情况

In [67]:
import json

def detect_constant_assignments(ipynb_file: str):
    with open(ipynb_file, 'r') as f:
        ipynb_data = json.load(f)

    # 遍历所有的代码单元
    for cell in ipynb_data['cells']:
        if cell['cell_type'] == 'code':
            # 遍历当前代码单元中的所有行
            for line in cell['source']:
                # 检测是否是赋值行，并获取变量名和值
                parts = line.split('=')
                if len(parts) == 2:
                    var_name = parts[0].strip()
                    value = parts[1].strip()
                    print(f'{var_name}: {value}')

detect_constant_assignments(updatedDataNotebookPath)


Model_Version: "V2.1-768px" #@param [ "1.5", "V2.1-512px", "V2.1-768px"]
Huggingface_Token: "" #@param {type:"string"}
token: Huggingface_Token
Custom_Model_Version: "1.5" #@param [ "1.5", "V2.1-512px", "V2.1-768px"]
Path_to_HuggingFace: "" #@param {type:"string"}
CKPT_Path: "" #@param {type:"string"}
CKPT_Link: "" #@param {type:"string"}
token: Huggingface_Token
token: input("Insert your huggingface token :")
if Path_to_HuggingFace !: "":
MODEL_NAME: "/root/content/stable-diffusion-custom"
MODEL_NAME: "/root/content/stable-diffusion-custom"
elif CKPT_Path !: "":
MODEL_NAME: "/root/content/stable-diffusion-custom"
elif CKPT_Link !: "":
MODEL_NAME: "/root/content/stable-diffusion-custom"
MODEL_NAME: "/root/content/stable-diffusion-v1-5"
MODEL_NAME: "/root/content/stable-diffusion-v1-5"
MODEL_NAME: "/root/content/stable-diffusion-v2-512"
MODEL_NAME: "/root/content/stable-diffusion-v2-512"
MODEL_NAME: "/root/content/stable-diffusion-v2-768"
MODEL_NAME: "/root/content/stable-diffusion-v2-7